In [1]:
from os import path

def cn(cadena):
    try:
        valor = float(cadena)  
        if valor.is_integer():
            return int(valor)  
        else:
            return valor  
    except ValueError:
        return cadena 
x = "db/Costos_producción.csv"

with open(x) as archivo:
    costos = archivo.readlines()
    costos = list(map(lambda x: x.strip().split(","), costos))
    costos = list(map(lambda x: [cn(x[0]), cn(x[1]), cn(x[2])], costos))
x = "db/movies.csv"

with open(x) as archivo:
    pelis = archivo.readlines()
    pelis = list(map(lambda x: x.strip().split(","), pelis))
    pelis = list(map(lambda lista: list(map(cn, lista)), pelis))
x = "db/ratings.csv"

with open(x) as archivo:
    ratings = archivo.readlines()
    ratings = list(map(lambda x: x.strip().split(","), ratings))
    ratings = list(map(lambda lista: list(map(cn, lista)), ratings))
x = "db/Recursos_new.csv"
with open(x) as archivo:
    recursos_new = archivo.readlines()
    recursos_new = list(map(lambda x: x.strip().split(","), recursos_new))
    recursos_new = list(map(lambda lista: list(map(cn, lista)), recursos_new))
x = "db/Recursos.csv"

with open(x) as archivo:
    recursos = archivo.readlines()
    recursos = list(map(lambda x: x.strip().split(","), recursos))
    recursos = list(map(lambda lista: list(map(cn, lista)), recursos))


In [2]:
class Vec:
    def __init__(self, dominio: list, coordenadas: dict) -> None:
        self.D = dominio

        if any(key not in self.D for key in coordenadas.keys()):
            raise ValueError("Hay ejes en cords que no estan en el domineo")

        self.f = coordenadas

    def __borrar_ceros(self):
        for k, v in deepcopy(self.f).items():
            if v == 0:
                del self.f[k]

    def getitem(self, k):
        if k not in self.D:
            raise ValueError(f"El eje {k} no existe en el domineo: {self.D}")
        return self.f.get(k, 0)

    def setitem(self, k, v):
        if k not in self.D:
            raise ValueError(f"El eje {k} no existe en el domineo: {self.D}")

        self.__borrar_ceros()

        self.f[k] = v

    def equal(self, u):
        return self.D == u.D and self.f == u.f

    def add(self, u):
        if self.D != u.D:
            raise ValueError("No tienen el mismo domineo")
        salida = Vec(deepcopy(self.D), deepcopy(self.f))

        for i in self.D:
            salida.setitem(i, self.getitem(i) + u.getitem(i))

        salida.__borrar_ceros()

        return salida

    def dot(self, u):
        if self.D != u.D:
            raise ValueError("No tienen el mismo domineo")
        r = 0

        union = set(self.f.keys()).union(u.f.keys())

        for i in union:
            r += self.getitem(i) * u.getitem(i)
        return r

    def scalar_mul(self, alpha):
        salida = Vec(deepcopy(self.D), deepcopy(self.f))
        for i in self.D:
            salida.setitem(i, salida.getitem(i) * alpha)

        salida.__borrar_ceros()

        return salida

    def neg(self):
        salida = Vec(deepcopy(self.D), deepcopy(self.f))
        for i in self.D:
            salida.setitem(i, salida.getitem(i) * -1)
        return salida

    def __str__(self) -> str:
        self.__borrar_ceros()
        return f"Vec({self.D},{self.f})"

    def __repr__(self) -> str:
        return f"Vec({len(self.D)},{self.f})"

In [3]:
dominio = []
users = {}
movies = {}


for _, movie_id, rating in ratings[1:]:
    dominio.append(movie_id)

usuarios = set(map(lambda x: x[0], ratings[1:]))
for user_id in usuarios:

    usurio_rating = list(filter(lambda x: x[0] == user_id, ratings[1:]))

    movie_id2rating = dict(map(lambda x: (x[1], x[2]), usurio_rating))
    vector_gusto = Vec(dominio=dominio, coordenadas=movie_id2rating)

    users[user_id] = vector_gusto

peliculas = set(map(lambda x: x[1], ratings[1:]))
for movie_id in peliculas:
    movie_id_rating = list(filter(lambda x: x[1] == movie_id, ratings[1:]))
    usur_id2rating = dict(map(lambda x: (x[0], x[2]), movie_id_rating))

    vector = Vec(dominio=usuarios, coordenadas=usur_id2rating)
    movies[movie_id] = vector

In [4]:
def sim(v: Vec, u: Vec):
    return (v.dot(u)) / (
        ( ((u.dot(u))**1/2)*((v.dot(v))**1/2 ) )
    )
def vecinos(users: dict, user_id: int, k: int):
    _users = users.copy()

    del _users[user_id]

    w = {user: sim(users[user_id], user_data) for user, user_data in _users.items()}

    w = [(user, sim(users.get(user_id), user_data)) for user, user_data in _users.items()]
    w = sorted(w, key=lambda x: x[1], reverse=True)

    return w[:k]

In [5]:
resultado_k = vecinos(users, 926, 30)
resultado_k

[(685, 0.009716599190283401),
 (740, 0.009709401709401709),
 (809, 0.009595526324150859),
 (105, 0.009482812402520432),
 (547, 0.009272697951943234),
 (482, 0.009141185611773846),
 (107, 0.00899165061014772),
 (866, 0.008649984553599012),
 (446, 0.008624008624008623),
 (431, 0.00848639146511487)]

In [6]:
def recomendacion(user_id):
    x = vecinos(users, user_id, 30)

    peliculas_usuario = list(filter(lambda x: x[0] == user_id, ratings[1:]))
    peliculas_usuario = list(map(lambda x: x[1], peliculas_usuario))

    pelicula_a_recomendar = []
    for user_id_vecino, similitud in x:
        peliculas_vecino = list(filter(lambda x: x[0] == user_id_vecino, ratings[1:]))

        for _, movie_id, *rest in peliculas_vecino:
            if len(pelicula_a_recomendar) >= 13:
                break
            if movie_id not in peliculas_usuario:
                pelicula_a_recomendar.append(movie_id)

    return pelicula_a_recomendar




In [7]:
recomendados = recomendacion(2)
recomendados

[245, 258, 262, 294, 321, 322, 325, 340, 1, 124, 326, 328, 333]

d) Se seleccionaron las recomendaciones según los 30 vecinos más cercanos, luego dentro de las peliculas no vistas por el usuario 
a consultar, es que se entregan en una lista las 13 peliculas en orden de búsqueda de prioridad por vecino. Se seleccionan los primeros 30 vecinos
en caso de que si hubieran demasiadas similaritudes de peliculas entre los primeros vecinos, exitiera la posibilidad de hacer una recomendación con 
aquellos que están más lejanos pero que aún pueden aportar películas no vistas. Se entregan 13 películas como opción al usuario para que pueda tener un 
espacio de selección no tan acotado como un top 3 pero con un poco más de opciones que un top 10. Sin dejar una cantidad muy amplia que dificulte la selección.
Por otro lado, es importante recalcar que se elige por orden las peliculas desde el vecino más cercano hasta que se acaben sus recomendaciones, 
ya que, permite tener las peliculas más cercanas en gustos al usuario. De esta forma hay posibilidades de que encuentre mejores produccciones que ver, 
teniendo en consideración que el promedio de los vecinos para recomendar podría no ser tan representativo como indicador.


e)Los beneficios de utilizar la clase que se programó anteriormente es que posee distintos métodos que facilitan la aplicación 
de operaciones dentro del problema planteado. Además del hecho que permite manejar aquellos casos en que no se tiene un rating, asumiéndose como cero.
Por ende detro del contexto, existen múltiples veces en que el usuario podría tener elementos con valor cero dentro del diccionario asignado. 
Es en ello que se clasifica como sparce data por la cantidad de datos con valor cero que existen. El manejo dentro de la clase no permite la existencia de operaciones,
fuera del dominio, haciendo que el modelo de recomendación funcione correctamente, arrojando error en caso de que sea necesario.